---
# Chapter 4
## Text versus Bytes
---

## Text versus Bytes

---
### Example 4-1: Encoding and decoding

In [ ]:
s = 'café'
print(s)
print(len(s))

b = s.encode('utf-8')
print(b)
print(len(b))

b.decode('utf-8')

---
### Example 4-2: Five-byte sequence as bytes and as bytearray

In [ ]:
cafe = bytes('café', encoding='utf_8')
print(cafe)

print(cafe[0])
print(cafe[:1])

In [ ]:
cafe_arr = bytearray(cafe)
print(cafe_arr)

cafe_arr[-1:]

In [ ]:
bytes.fromhex('31 4B CE A9')

---
### Example 4-3: Initializing bytes from the raw data of an array

In [ ]:
import array

numbers = array.array('h', [-2, -1, 0, 1, 2])
octets = bytes(numbers)
print(octets)

---
### Example 4-4:  Using memoryview and struc to inspec a GIF image header

In [ ]:
import struct

fmt = '<3s3sHH'

with open('sample.gif', 'rb') as fp:
    img = memoryview(fp.read())

header = img[:10]
header_bytes = bytes(header)
print(header_bytes)

unpacked = struct.unpack(fmt, header)
print(unpacked)

del header
del header_bytes
del unpacked
del img

---
### Example 4-5: The string "El Niño" encoded with three codecs producing very different bytes sequences

In [ ]:

s = 'El Niño'
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, s.encode(codec), sep='\t')

---
### Example 4-6: Encodign to bytes: success and error handling

In [ ]:
city = 'São Paulo'
city.encode('utf-8')

In [ ]:
city.encode('utf-16')

In [ ]:
city.encode('iso8859_1')

In [ ]:
city.encode('cp437')

In [ ]:
city.encode('cp437', errors='ignore')

In [ ]:
city.encode('cp437', errors='replace')

In [ ]:
city.encode('cp437', errors='xmlcharrefreplace')

---
### Example 4-7: Decoding from str to bytes: succes and error handling

In [ ]:
octets = b'Mont\xe9al'
octets.decode('cp1252')

In [ ]:
octets.decode('iso8859_7')

In [ ]:
octets.decode('koi8_r')

In [ ]:
octets.decode('utf-8')

In [ ]:
octets.decode('utf-8', errors='replace')

---
### Example 4-9: A platform encoding issue (if you try this in your machine, ypu may or may not see the problem)

In [ ]:
open('cafe.txt', 'w', encoding='utf-8').write('café')

open('cafe.txt').read()

---
### Example 4-10: Closer inspection of example 4-9 running on Windows reveals the bug and how fix it.

In [ ]:
fp = open('cafe.txt', 'w', encoding='utf-8')
fp

In [ ]:
fp.write('café')

In [ ]:
fp.close()

In [ ]:
import os
os.stat('cafe.txt').st_size

In [ ]:
fp2 = open('cafe.txt')
fp2

---
### Example 4-11: Exploring encoding defaults

In [ ]:
import sys, locale

expressions= """
    locale.getpreferredencoding()
    type(myfile)
    myfile.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
    
"""
myfile = open('dummy', 'w')

for expresion in expressions.split():
    value = eval(expresion)
    print(expresion.rjust(30), '->', repr(value))

## Normalizing Unicode for Saner Comparisons

In [ ]:
s1 = 'café'
s2 = 'cafe\u0301'

s1, s2

In [ ]:
len(s1), len(s2)

In [ ]:
s1 == s2

In [ ]:
from unicodedata import normalize

s1 = 'café'
s2 = 'cafe\u0301'

print(len(s1), len(s2))

ns1 = normalize('NFC', s1)
ns2 = normalize('NFC', s2)
print(len(ns1), len(ns2))
print(ns1 == ns2)

n2s1 = normalize('NFD', s1)
n2s2 = normalize('NFD', s2)
print(len(n2s1), len(n2s2))
print(ns1 == ns2)


In [ ]:
from unicodedata import normalize, name

ohm = '\u2126'
print(name(ohm))

ohm_c = normalize('NFC', ohm)
print(name(ohm_c))

print(ohm == ohm_c)

normalize('NFC', ohm) == normalize('NFC', ohm_c)

---
### Example 4-14: Function to remove all combining marks.

In [ ]:
import unicodedata
import string

def shave_marks(txt):
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

---
### Example 4-15: Two examples using shave_marks from Example 4-14

In [ ]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

In [ ]:
greek = 'Ζέφυρος, Zéfiro'
shave_marks(greek)

---
### Example 4-16: Function to remove combining marks from Latin characters.

In [ ]:
def shave_marks_latin(txt):
    norm_txt = unicodedata.normalize('NFD', txt)
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue

        keepers.append(c)
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters

    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)

In [ ]:
shave_marks_latin(order)

In [ ]:
shave_marks(order) == shave_marks_latin(order)

---
### Example 4-17: Transform some Western typographical symbols into ASCII

In [ ]:
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)

def dewinize(txt):
    return txt.translate(multi_map)

def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))
    no_marks = no_marks.replace('ß', 'ss')
    return unicodedata.normalize('NFC', no_marks)

In [ ]:
dewinize(order)

In [ ]:
dewinize(greek)

---
### Example 4-18: Two examples using asciize from Example 4-17

In [ ]:
asciize(order)

In [ ]:
asciize(greek)

## Sorting Unicode Text

In [ ]:
fruits = ['caju', 'atemoia', 'cajá', 'acai', 'acerola']

sorted(fruits)

---
### Example 4-19: Using the locale.strxfrm function as sort key

In [ ]:
import locale

locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')

fruits = ['caju', 'atemoia', 'cajá', 'acai', 'acerola']

sorted_fruits = sorted(fruits, key=locale.strxfrm)
print(sorted_fruits)

## Sorting with the Unicode Collation Algorithm

---
### Example 4-20: Using the pyuca.Collator.sort_key method

In [ ]:
import pyuca

coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'acai', 'acerola']
sorted_fruits = sorted(fruits, key=coll.sort_key)
print(sorted_fruits)

## The Unicode Database

---
### Example 4-21: Demo of Unicode daabase numerical character metadata (callouts describe each column in the output)

In [ ]:
import unicodedata
import re

re_digit = re.compile('\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),
        char.center(6),
        're_dig' if re_digit.match(char) else '-',
        'isdigit' if char.isdigit() else '-',
        'isnum' if char.isnumeric() else '-',
        format(unicodedata.numeric(char), '5.2f'),
        unicodedata.name(char),
        sep='\t'
    )
    

---
### Example 4-22: Compare behavior of simple str and bytes regular expressions

In [ ]:
import re

re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
            " as 1729 = 1³ + 12³ = 9³ + 10³.")

text_bytes = text_str.encode('utf-8')
print('Text', repr(text_str), sep='\n')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))
print('  bytes  :', re_numbers_bytes.findall(text_bytes))
print('Words')
print('  str  :', re_words_str.findall(text_str))
print('  bytes  :', re_words_bytes.findall(text_bytes))